# Building Prediction Model

## Implort to be used packeges and module

In [1]:
import pandas as pd 
import sklearn
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from models.schemas import Base, CurrentWeather, StaticBike
from config.config import MySQL
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import pickle

## Load data

### Connect to database

In [2]:
try:
    db = create_engine(MySQL.URI)
    conn = db.connect()
except Exception as e:
    # Close connections
    conn.close()
    db.dispose()

### Retrieve data and create dataframe

In [3]:
df_all_data = pd.read_sql_query("SELECT * FROM dublin_bike.bike_history AS b INNER JOIN dublin_bike.weather_history AS w ON b.scraping_time = w.datetime and b.number = w.stationNum", conn)
df_all_data.head(5)

,scraping_time,number,last_update,address,site_names,latitude,longitude,bike_stand,available_bike_stand,available_bike,...,lon,lat,wind_spd,clouds,sunset,sunrise,pressure,humidity,code,weekday
0,2021-03-01 12:35:29,2,2021-03-01 12:27:15,Blessington Street,BLESSINGTON STREET,53.3568,-6.26814,20,19,0,...,-6.2681,53.3568,3.6,75.0,1614621749,1614621749,1031,87,701,1
1,2021-03-01 12:35:29,3,2021-03-01 12:27:13,Bolton Street,BOLTON STREET,53.3512,-6.26986,20,13,7,...,-6.2699,53.3512,3.6,75.0,1614621750,1614621750,1031,87,701,1
2,2021-03-01 12:35:29,4,None,Greek Street,GREEK STREET,53.3469,-6.27298,20,0,0,...,-6.2699,53.3512,3.6,75.0,1614621750,1614621750,1031,87,701,1
3,2021-03-01 12:35:29,5,2021-03-01 12:28:02,Charlemont Street,CHARLEMONT PLACE,53.3307,-6.26018,40,16,24,...,-6.2602,53.3307,3.6,75.0,1614621749,1614621749,1031,87,701,1
4,2021-03-01 12:35:29,6,2021-03-01 12:34:02,Christchurch Place,CHRISTCHURCH PLACE,53.3434,-6.27012,20,15,5,...,-6.2701,53.3434,3.6,75.0,1614621750,1614621750,1031,87,701,1


In [4]:
df_all_data.shape

(648216, 28)

### Change scrapping_time to datetime and create the hour and minute column

In [5]:
df_all_data['scraping_time'] = pd.to_datetime(df_all_data['scraping_time'])
df_all_data['hour'] = df_all_data['scraping_time'].dt.hour
df_all_data['minutes'] = df_all_data['scraping_time'].dt.minute
df_all_data

,scraping_time,number,last_update,address,site_names,latitude,longitude,bike_stand,available_bike_stand,available_bike,...,wind_spd,clouds,sunset,sunrise,pressure,humidity,code,weekday,hour,minutes
0,2021-03-01 12:35:29,2,2021-03-01 12:27:15,Blessington Street,BLESSINGTON STREET,53.3568,-6.26814,20,19,0,...,3.60,75.0,1614621749,1614621749,1031,87,701,1,12,35
1,2021-03-01 12:35:29,3,2021-03-01 12:27:13,Bolton Street,BOLTON STREET,53.3512,-6.26986,20,13,7,...,3.60,75.0,1614621750,1614621750,1031,87,701,1,12,35
2,2021-03-01 12:35:29,4,None,Greek Street,GREEK STREET,53.3469,-6.27298,20,0,0,...,3.60,75.0,1614621750,1614621750,1031,87,701,1,12,35
3,2021-03-01 12:35:29,5,2021-03-01 12:28:02,Charlemont Street,CHARLEMONT PLACE,53.3307,-6.26018,40,16,24,...,3.60,75.0,1614621749,1614621749,1031,87,701,1,12,35
4,2021-03-01 12:35:29,6,2021-03-01 12:34:02,Christchurch Place,CHRISTCHURCH PLACE,53.3434,-6.27012,20,15,5,...,3.60,75.0,1614621750,1614621750,1031,87,701,1,12,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648211,2021-03-23 10:10:04,112,2021-03-23 10:06:17,North Circular Road (O'Connell's),NORTH CIRCULAR ROAD (O'CONNELL'S),53.3578,-6.25156,30,29,1,...,6.17,75.0,1616524983,1616524983,1018,66,803,2,10,10
648212,2021-03-23 10:10:04,113,2021-03-23 10:08:53,Merrion Square South,MERRION SQUARE SOUTH,53.3386,-6.24861,40,30,10,...,6.17,75.0,1616524983,1616524983,1018,66,803,2,10,10
648213,2021-03-23 10:10:04,114,2021-03-23 10:06:42,Wilton Terrace (Park),WILTON TERRACE (PARK),53.3337,-6.24834,40,28,12,...,6.17,75.0,1616524981,1616524981,1018,66,803,2,10,10
648214,2021-03-23 10:10:04,115,2021-03-23 10:08:38,Killarney Street,KILLARNEY STREET,53.3548,-6.24758,30,21,9,...,6.17,75.0,1616524983,1616524983,1018,66,803,2,10,10


### Transform week data 

In [6]:
#use number to replace weekdays
df_all_data['weekday'] = df_all_data['weekday'].replace(1, 'Monday')
df_all_data['weekday'] = df_all_data['weekday'].replace(2, 'Tuesday')
df_all_data['weekday'] = df_all_data['weekday'].replace(3, 'Wednesday')
df_all_data['weekday'] = df_all_data['weekday'].replace(4, 'Thursday')
df_all_data['weekday'] = df_all_data['weekday'].replace(5, 'Friday')
df_all_data['weekday'] = df_all_data['weekday'].replace(6, 'Saturday')
df_all_data['weekday'] = df_all_data['weekday'].replace(7, 'Sunday')
df_all_data

,scraping_time,number,last_update,address,site_names,latitude,longitude,bike_stand,available_bike_stand,available_bike,...,wind_spd,clouds,sunset,sunrise,pressure,humidity,code,weekday,hour,minutes
0,2021-03-01 12:35:29,2,2021-03-01 12:27:15,Blessington Street,BLESSINGTON STREET,53.3568,-6.26814,20,19,0,...,3.60,75.0,1614621749,1614621749,1031,87,701,Monday,12,35
1,2021-03-01 12:35:29,3,2021-03-01 12:27:13,Bolton Street,BOLTON STREET,53.3512,-6.26986,20,13,7,...,3.60,75.0,1614621750,1614621750,1031,87,701,Monday,12,35
2,2021-03-01 12:35:29,4,None,Greek Street,GREEK STREET,53.3469,-6.27298,20,0,0,...,3.60,75.0,1614621750,1614621750,1031,87,701,Monday,12,35
3,2021-03-01 12:35:29,5,2021-03-01 12:28:02,Charlemont Street,CHARLEMONT PLACE,53.3307,-6.26018,40,16,24,...,3.60,75.0,1614621749,1614621749,1031,87,701,Monday,12,35
4,2021-03-01 12:35:29,6,2021-03-01 12:34:02,Christchurch Place,CHRISTCHURCH PLACE,53.3434,-6.27012,20,15,5,...,3.60,75.0,1614621750,1614621750,1031,87,701,Monday,12,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648211,2021-03-23 10:10:04,112,2021-03-23 10:06:17,North Circular Road (O'Connell's),NORTH CIRCULAR ROAD (O'CONNELL'S),53.3578,-6.25156,30,29,1,...,6.17,75.0,1616524983,1616524983,1018,66,803,Tuesday,10,10
648212,2021-03-23 10:10:04,113,2021-03-23 10:08:53,Merrion Square South,MERRION SQUARE SOUTH,53.3386,-6.24861,40,30,10,...,6.17,75.0,1616524983,1616524983,1018,66,803,Tuesday,10,10
648213,2021-03-23 10:10:04,114,2021-03-23 10:06:42,Wilton Terrace (Park),WILTON TERRACE (PARK),53.3337,-6.24834,40,28,12,...,6.17,75.0,1616524981,1616524981,1018,66,803,Tuesday,10,10
648214,2021-03-23 10:10:04,115,2021-03-23 10:08:38,Killarney Street,KILLARNEY STREET,53.3548,-6.24758,30,21,9,...,6.17,75.0,1616524983,1616524983,1018,66,803,Tuesday,10,10


In [7]:
df_all_data['weekday'].unique()

array(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday'], dtype=object)

In [8]:
# Create a separate dataframe with days of the week (categorical)
data_input = pd.DataFrame(df_all_data['weekday'])

# Concatenate the two dataframes in the main one
dummy = pd.get_dummies(data_input)
df_all_data = pd.concat([df_all_data,dummy],axis=1)
df_all_data

,scraping_time,number,last_update,address,site_names,latitude,longitude,bike_stand,available_bike_stand,available_bike,...,weekday,hour,minutes,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,2021-03-01 12:35:29,2,2021-03-01 12:27:15,Blessington Street,BLESSINGTON STREET,53.3568,-6.26814,20,19,0,...,Monday,12,35,0,1,0,0,0,0,0
1,2021-03-01 12:35:29,3,2021-03-01 12:27:13,Bolton Street,BOLTON STREET,53.3512,-6.26986,20,13,7,...,Monday,12,35,0,1,0,0,0,0,0
2,2021-03-01 12:35:29,4,None,Greek Street,GREEK STREET,53.3469,-6.27298,20,0,0,...,Monday,12,35,0,1,0,0,0,0,0
3,2021-03-01 12:35:29,5,2021-03-01 12:28:02,Charlemont Street,CHARLEMONT PLACE,53.3307,-6.26018,40,16,24,...,Monday,12,35,0,1,0,0,0,0,0
4,2021-03-01 12:35:29,6,2021-03-01 12:34:02,Christchurch Place,CHRISTCHURCH PLACE,53.3434,-6.27012,20,15,5,...,Monday,12,35,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648211,2021-03-23 10:10:04,112,2021-03-23 10:06:17,North Circular Road (O'Connell's),NORTH CIRCULAR ROAD (O'CONNELL'S),53.3578,-6.25156,30,29,1,...,Tuesday,10,10,0,0,0,0,0,1,0
648212,2021-03-23 10:10:04,113,2021-03-23 10:08:53,Merrion Square South,MERRION SQUARE SOUTH,53.3386,-6.24861,40,30,10,...,Tuesday,10,10,0,0,0,0,0,1,0
648213,2021-03-23 10:10:04,114,2021-03-23 10:06:42,Wilton Terrace (Park),WILTON TERRACE (PARK),53.3337,-6.24834,40,28,12,...,Tuesday,10,10,0,0,0,0,0,1,0
648214,2021-03-23 10:10:04,115,2021-03-23 10:08:38,Killarney Street,KILLARNEY STREET,53.3548,-6.24758,30,21,9,...,Tuesday,10,10,0,0,0,0,0,1,0


In [9]:
df_all_data.columns

Index(['scraping_time', 'number', 'last_update', 'address', 'site_names',
       'latitude', 'longitude', 'bike_stand', 'available_bike_stand',
       'available_bike', 'status', 'banking', 'bonus', 'localtime',
       'stationNum', 'datetime', 'temperature', 'icon', 'lon', 'lat',
       'wind_spd', 'clouds', 'sunset', 'sunrise', 'pressure', 'humidity',
       'code', 'weekday', 'hour', 'minutes', 'weekday_Friday',
       'weekday_Monday', 'weekday_Saturday', 'weekday_Sunday',
       'weekday_Thursday', 'weekday_Tuesday', 'weekday_Wednesday'],
      dtype='object')

In [10]:
df_all_data.dtypes

scraping_time           datetime64[ns]
number                           int64
last_update                     object
address                         object
site_names                      object
latitude                       float64
longitude                      float64
bike_stand                       int64
available_bike_stand             int64
available_bike                   int64
status                          object
banking                          int64
bonus                            int64
localtime               datetime64[ns]
stationNum                       int64
datetime                datetime64[ns]
temperature                     object
icon                            object
lon                            float64
lat                            float64
wind_spd                       float64
clouds                         float64
sunset                          object
sunrise                         object
pressure                        object
humidity                 

## Create a prediction model for available bikes

### Select input and output features

In [11]:
input_model = pd.DataFrame(df_all_data[['latitude', 'longitude','temperature','wind_spd', 'pressure', 'humidity', 'hour']])
input_model = pd.concat([input_model,dummy], axis = 1)
output = df_all_data['available_bike']

In [12]:
input_model

,latitude,longitude,temperature,wind_spd,pressure,humidity,hour,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,53.3568,-6.26814,7.37,3.60,1031,87,12,0,1,0,0,0,0,0
1,53.3512,-6.26986,7.37,3.60,1031,87,12,0,1,0,0,0,0,0
2,53.3469,-6.27298,7.37,3.60,1031,87,12,0,1,0,0,0,0,0
3,53.3307,-6.26018,7.38,3.60,1031,87,12,0,1,0,0,0,0,0
4,53.3434,-6.27012,7.37,3.60,1031,87,12,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648211,53.3578,-6.25156,8.53,6.17,1018,66,10,0,0,0,0,0,1,0
648212,53.3386,-6.24861,8.54,6.17,1018,66,10,0,0,0,0,0,1,0
648213,53.3337,-6.24834,8.55,6.17,1018,66,10,0,0,0,0,0,1,0
648214,53.3548,-6.24758,8.54,6.17,1018,66,10,0,0,0,0,0,1,0


In [13]:
output

0          0
1          7
2          0
3         24
4          5
          ..
648211     1
648212    10
648213    12
648214     9
648215     0
Name: available_bike, Length: 648216, dtype: int64

### Split dataset to train and test 

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.2,random_state=40)
print("Will train the model on %s rows and %s columns." % X_train.shape)
print("Will test the model on %s rows and %s columns." % X_test.shape)

Will train the model on 518572 rows and 14 columns.
Will test the model on 129644 rows and 14 columns.


### Train the model

In [15]:
# Instantiate RandomForestRegressor object (calling 10 decision tree models)
model = RandomForestRegressor(n_estimators = 10)

# Train the model
model.fit(X_train, Y_train)

RandomForestRegressor(n_estimators=10)

### Test the model

In [16]:
prediction = model.predict(X_test)

In [17]:
#make a new datafram to show the predicted available bikes
df_prediction = pd.DataFrame(prediction, columns=['Predicted'])

#convert all the data for testing to a new datafram
df_test = df_all_data.iloc[Y_test]

#reset the index
df_bikes = pd.DataFrame(df_test['available_bike']).reset_index(drop=True)

#to get a clear comparisaon, concatenate two new datafram
actual_vs_predicted= pd.concat([df_bikes,df_prediction], axis=1)
actual_vs_predicted['difference'] = actual_vs_predicted['Predicted'] - actual_vs_predicted['available_bike']
actual_vs_predicted

,available_bike,Predicted,difference
0,3,10.000000,7.000000
1,12,11.000000,-1.000000
2,6,12.000000,6.000000
3,0,0.000000,0.000000
4,12,11.000000,-1.000000
...,...,...,...
129639,24,14.700000,-9.300000
129640,7,18.200000,11.200000
129641,12,11.186667,-0.813333
129642,18,29.000000,11.000000


### Evaluation

In [18]:
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print("Mean Absolute Error (MAE): ", metrics.mean_absolute_error(testActualVal, predictions))
    print("Mean Squared Error (MSE): ", metrics.mean_squared_error(testActualVal, predictions))
    print("Root Mean Squared Error (RMSE): ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))
printMetrics(Y_test, prediction)

Mean Absolute Error (MAE):  0.3230246968179113
Mean Squared Error (MSE):  0.5960962688400622
Root Mean Squared Error (RMSE):  0.7720727095552997
R2:  0.9874216948396197


In [19]:
pickle.dump(model,open('flask_app/bike_prediction_model.pickle', 'wb'))

## Create prediction model for available stands

### Select input and output features

In [20]:
input_model = pd.DataFrame(df_all_data[['latitude', 'longitude','temperature','wind_spd', 'pressure', 'humidity', 'hour']])
input_model = pd.concat([input_model,dummy],axis=1)
output = df_all_data['available_bike_stand']

### Split dataset to train and test 

In [21]:
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.2,random_state=40)
print("Will train the model on %s rows and %s columns." % X_train.shape)
print("Will test the model on %s rows and %s columns." % X_test.shape)

Will train the model on 518572 rows and 14 columns.
Will test the model on 129644 rows and 14 columns.


### Train the model

In [22]:
# Instantiate RandomForestRegressor object calling 10 decision tree models
model = RandomForestRegressor(n_estimators=10)

# Train the model
model.fit(X_train, Y_train)

RandomForestRegressor(n_estimators=10)

In [23]:
prediction = model.predict(X_test)

In [24]:
#make a new datafram to show the predicted available bikes
df_prediction = pd.DataFrame(prediction, columns=['Predicted'])

#convert all the data for testing to a new datafram
df_test = df_all_data.iloc[Y_test]

#reset the index
df_bikes = pd.DataFrame(df_test['available_bike_stand']).reset_index(drop=True)

#to get a clear comparisaon, concatenate two new datafram
actual_vs_predicted= pd.concat([df_bikes,df_prediction], axis=1)
actual_vs_predicted['difference'] = actual_vs_predicted['Predicted'] - actual_vs_predicted['available_bike_stand']
actual_vs_predicted

,available_bike_stand,Predicted,difference
0,22,30.00,8.00
1,13,18.00,5.00
2,4,8.00,4.00
3,21,40.00,19.00
4,29,25.00,-4.00
...,...,...,...
129639,6,23.10,17.10
129640,18,10.70,-7.30
129641,5,28.46,23.46
129642,18,11.02,-6.98


### Evaluation

In [25]:
printMetrics(Y_test, prediction)

Mean Absolute Error (MAE):  0.3246505404900556
Mean Squared Error (MSE):  0.6961063426614297
Root Mean Squared Error (RMSE):  0.8343298764046686
R2:  0.9917785159540783


In [26]:
pickle.dump(model,open('flask_app/stand_prediction_model.pickle', 'wb'))

In [27]:
conn.close()
db.dispose()